# Exploratory analysis on the Paris food scene

This project is a statistical analysis on restaurants in Paris, France, using YELP API data. I used the YELP API to scrape summary (40.000m within the google search, "Paris, France") and collected data for 12000 restaurants.